# 2.0

In [1]:
from rtree import index
import numpy as np
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [153]:
def initiate(radius,C,L):
    coordinate = np.ones(2)*L/2.0
    nodelist = {(coordinate[0],coordinate[1]):{5:0.0,7:1}} #5：到心距 7：点数
    index2coor = {1:(coordinate[0],coordinate[1])}
    idx = index.Index()
    idx.insert(1,list(np.r_[coordinate,coordinate]))                     
    limitt = np.r_[coordinate - radius,coordinate + radius] 
    return nodelist,idx,limitt,index2coor

def choosenode(nodelist,limitt,C,index2coor):#?
    white = ( (limitt[2]-limitt[0])*(limitt[3]-limitt[1])-len(nodelist) )*C
    exist = len(index2coor)+len(nodelist)*C
    rnd = random.random()*( exist + white )
    if rnd>exist:
        while 1:
            ii = int( random.random()*(limitt[2]-limitt[0])   + limitt[0] )
            jj = int( random.random()*(limitt[3]-limitt[1])   + limitt[1] )
            if (ii,jj) not in nodelist:
                return (ii,jj),0
    else:
        pos = random.randint(1,len(index2coor))#右闭
        return index2coor[pos],1

def onestep(nodelist,index2coor,idx,radius,C,h,limitt,L):
    two,flag = choosenode(nodelist,limitt,C,index2coor)  
    i = two[0]
    j = two[1]
    if flag == 0:
        intersection = list(idx.intersection([i-radius,j-radius, i+radius,j+radius])) 
        if len(intersection) > 0:
            for key in intersection:
                if np.linalg.norm(np.array([index2coor[key]]) - np.array([i,j])) <= radius:  
                    state = 1
                else:
                    state = 0
            if state == 1:
                nodelist,index2coor,idx,limitt = insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L)
        return nodelist,index2coor,idx,limitt      

    elif flag==1: 
        if nodelist[(i,j)][7]+1 <= h:#不溢
            nodelist[(i,j)][7] += 1
            index2coor[len(index2coor)+1]=(i,j)
        else:
            while ((i,j) in nodelist.keys()) and (nodelist[(i,j)][7]+1 > h) :
                (i,j) = sandpile(i,j,h,nodelist)
            if (i,j) in nodelist.keys():#溢出到有点的格子
                nodelist[(i,j)][7] += 1
                index2coor[len(index2coor)+1]=(i,j)
            else:#溢出到空格子
                nodelist,index2coor,idx,limitt  = insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L)
        return nodelist,index2coor,idx,limitt
		
def insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L):
	nodelist[(i,j)] = {1:np.linalg.norm(np.array([i,j])-L/2),2:1}
	index2coor[len(index2coor)+1]=(i,j)
	newpoint = [i,j]
	idx.insert( len(index2coor), np.r_[newpoint, newpoint] )  ###idx
	for x in range(2):
		limitt[x]   = round(min(limitt[x],newpoint[x]-radius))
		limitt[x+2] = round(max(limitt[x+2],newpoint[x]+radius))
	return nodelist,index2coor,idx,limitt

def sandpile(i,j,h,nodelist):
    r = i
    c = j
    arounddic = {
	1:(r-1,c-1),2:(r -1,c),3:(r -1,c+1),
	4:(r,c-1),5:(r,c),6:(r,c+1),
	7:(r+1,c-1),8:(r+1,c),9:(r+1,c+1)
	}
    deltas = []
    #邻接阵arov
    arov={}
    for m in range(1,10):
        if arounddic[m] in nodelist:
            arov[m] = nodelist[arounddic[m]][7]
        else:
            arov[m] = 0
        delta =nodelist[(r,c)][7] - arov[m] 
        deltas.append(delta)
    B = np.cumsum(deltas)
    if B[-1]!= 0:
        rnd = random.random()*B[-1]
        ind = np.nonzero(B<rnd)
        if len(ind[0])==0:
            pos = 1
        else:
            pos = ind[0][-1]+2
    else:#九格同
        pos = 5 
        while pos == 5:
            pos = random.randint(1,9)
		#pos = 1 #左下
		#pos = 9 #右上
    #得到位置
    return arounddic[pos]        

def caculate(nodelist,radius):
    w = []
    delta  = radius
	#delta = radius/2#qqq
    for key in nodelist:
        w.append([nodelist[key][5],nodelist[key][7]])
    w.sort()
    w = np.array(w)
    R = np.arange(0,np.max(w[:,0]),delta)
    RL = np.zeros((len(R),2)) 
    RL[:,0] = R
    for i in range(len(RL)):
        RL[i,1] = np.sum(w[w[:,0]<RL[i,0],1])
    Rrho = np.zeros((len(R),2))
    Rrho[:,0] = R + delta/2
    for i in range(len(Rrho)-1):
        Rrho[i,1] = (RL[i+1,1] - RL[i,1])/(math.pi*(delta**2+2*delta*RL[i,0]))
    a = Rrho
    xx = np.nonzero(a[:,1]==0) 
    if len(xx[0])==0:
        x = 0 
    else:
        x = xx[0][0]
    Rrhox = Rrho[:x,:x]
    return Rrhox

def save(Rx,C,radius,maxnode,h=3,i=1):#i 预留次数参数
    se = {}
    for i in range(1,len(Rx)+1):
        se[i] = pd.Series(Rx[i])
    df = pd.concat([se[i] for i in range(1,len(Rx)+1) ],axis = 1)
    name = "C"+str(C)+"--r"+str(radius)+"--maxnode"+str(maxnode)+"--h"+str(h)+"--i"+str(i)
    df.to_csv(name+".csv")
    print("Winner winner chicken dinner")

def side_view(nodelist):
    h = []
    for k in nodelist:
        h.append(nodelist[k][7])
    a = Counter(h)
    print(a)
	
def spatial_range(nodelist):
    x = []
    y = []
    for k in nodelist.keys():
        x.append(k[0])
        y.append(k[1])
    return max(x),min(x),max(y),min(y)

def paint(nodelist):
	x = nodelist
	a = []
	m = []
	n = []
	c = []
	xmax,xmin,ymax,ymin = spatial_range(x)
	for k,v in x.items():
		a.append(k)
		c.append(1/(v[7]))
	for i in a:
		m.append(i[0])
		n.append(i[1])
	plt.figure(figsize=(10,10),dpi=100)
	plt.scatter(m,n,s = 0.1,c = c)
	plt.xlim((xmin,xmax))
	plt.ylim((ymin,ymax))
	plt.show()

# 3.0

In [154]:
from rtree import index
import numpy as np
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

## city

In [157]:
def initiate(L,radius):
    '''根据L初始化格网
    返回:idx:rtree索引,limitt:限制矩形范围,index2coor:存储生成顺序
    '''
    coordinate = np.ones(2)*L/2.0
    index2coor = [(coordinate[0],coordinate[1])]
    idx = index.Index()
    idx.insert(1,list(np.r_[coordinate,coordinate]))                
    limitt = np.r_[coordinate - radius,coordinate + radius]
    nodelist = {(coordinate[0],coordinate[1]):{1:0.0,1:1}}
    return nodelist,idx,limitt,index2coor
def choosenode(nodelist,limitt,index2coor,C):
    white = ( (limitt[2]-limitt[0])*(limitt[3]-limitt[1])-len(nodelist) )*C
    exist = len(index2coor)+len(nodelist)*C
    rnd = random.random()*( exist + white )
    if rnd>exist:
        while 1:
            ii = int( random.random()*(limitt[2]-limitt[0])   + limitt[0] )
            jj = int( random.random()*(limitt[3]-limitt[1])   + limitt[1] )
            if (ii,jj) not in nodelist:
                return (ii,jj),0
    else:
        pos = random.randint(1,len(index2coor))#右闭
        return index2coor[pos],1

def onestep(nodelist,idx,limitt,index2coor,L,radius,C,h):
    two,flag = choosenode(nodelist,limitt,index2coor,C)  
    i,j = two[0],two[1]
    if flag == 0:
        intersection = list(idx.intersection([i-radius,j-radius, i+radius,j+radius])) 
        if len(intersection) > 0:
            for key in intersection:
                if np.linalg.norm(np.array([index2coor[key]]) - np.array([i,j])) <= radius:  
                    state = 1
                else:
                    state = 0
            if state == 1:
                nodelist,index2coor,idx,limitt = insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L)
        return nodelist,index2coor,idx,limitt      

    elif flag==1: 
        if nodelist[(i,j)][7]+1 <= h:#不溢
            nodelist[(i,j)][7] += 1
            index2coor[len(index2coor)+1]=(i,j)
        else:
            while ((i,j) in nodelist.keys()) and (nodelist[(i,j)][7]+1 > h) :
                (i,j) = sandpile(i,j,h,nodelist)
            if (i,j) in nodelist.keys():#溢出到有点的格子
                nodelist[(i,j)][7] += 1
                index2coor[len(index2coor)+1]=(i,j)
            else:#溢出到空格子
                nodelist,index2coor,idx,limitt  = insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L)
        return nodelist,idx,limitt,index2coor
    
class City(object):
    def __init__(self,maxP,H,C,L = 10**5,radius = 10):
        '''初始全局参数
        可变参数：maxP:最大P,H:沙堆阈值,C:引力c
        默认参数：L:边框半径,radius:作用半径
        '''
        self.maxP = maxP
        self.H,self.C,self.L= H,C,L
        self.radius = radius
        
    def step(self,p):
        if p == 1:
            return initiate(self.L,self.radius)
        else:
            nodelist,idx,limitt,index2coor=City.step(self,p-1)
            return onestep(nodelist,idx,limitt,index2coor,self.L,self.radius,self.C,self.H)

In [158]:
c = City(2,3,0.05)
c.step(c.maxP)

IndexError: list index out of range

## demo1

In [162]:
def initiate(radius,C,L):
    coordinate = np.ones(2)*L/2.0
    nodelist = {(coordinate[0],coordinate[1]):{5:0.0,7:1}} #5：到心距 7：点数
    index2coor = {1:(coordinate[0],coordinate[1])}
    idx = index.Index()
    idx.insert(1,list(np.r_[coordinate,coordinate]))                     
    limitt = np.r_[coordinate - radius,coordinate + radius] 
    return nodelist,idx,limitt,index2coor

def choosenode(nodelist,limitt,C,index2coor):#?
    white = ( (limitt[2]-limitt[0])*(limitt[3]-limitt[1])-len(nodelist) )*C
    exist = len(index2coor)+len(nodelist)*C
    rnd = random.random()*( exist + white )
    if rnd>exist:
        while 1:
            ii = int( random.random()*(limitt[2]-limitt[0])   + limitt[0] )
            jj = int( random.random()*(limitt[3]-limitt[1])   + limitt[1] )
            if (ii,jj) not in nodelist:
                return (ii,jj),0
    else:
        pos = random.randint(1,len(index2coor))#右闭
        return index2coor[pos],1

def onestep(nodelist,index2coor,idx,radius,C,h,limitt,L):
    two,flag = choosenode(nodelist,limitt,C,index2coor)  
    i = two[0]
    j = two[1]
    if flag == 0:
        intersection = list(idx.intersection([i-radius,j-radius, i+radius,j+radius])) 
        if len(intersection) > 0:
            for key in intersection:
                if np.linalg.norm(np.array([index2coor[key]]) - np.array([i,j])) <= radius:  
                    state = 1
                else:
                    state = 0
            if state == 1:
                nodelist,index2coor,idx,limitt = insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L)
        return nodelist,index2coor,idx,limitt      

    elif flag==1: 
        if nodelist[(i,j)][7]+1 <= h:#不溢
            nodelist[(i,j)][7] += 1
            index2coor[len(index2coor)+1]=(i,j)
        else:
            while ((i,j) in nodelist.keys()) and (nodelist[(i,j)][7]+1 > h) :
                (i,j) = sandpile(i,j,h,nodelist)
            if (i,j) in nodelist.keys():#溢出到有点的格子
                nodelist[(i,j)][7] += 1
                index2coor[len(index2coor)+1]=(i,j)
            else:#溢出到空格子
                nodelist,index2coor,idx,limitt  = insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L)
        return nodelist,index2coor,idx,limitt
		
def insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L):
	nodelist[(i,j)] = {5:np.linalg.norm(np.array([i,j])-L/2),7:1}
	index2coor[len(index2coor)+1]=(i,j)
	newpoint = [i,j]
	idx.insert( len(index2coor), np.r_[newpoint, newpoint] )  ###idx
	for x in range(2):
		limitt[x]   = round(min(limitt[x],newpoint[x]-radius))
		limitt[x+2] = round(max(limitt[x+2],newpoint[x]+radius))
	return nodelist,index2coor,idx,limitt

def sandpile(i,j,h,nodelist):
    r = i
    c = j
    arounddic = {
	1:(r-1,c-1),2:(r -1,c),3:(r -1,c+1),
	4:(r,c-1),5:(r,c),6:(r,c+1),
	7:(r+1,c-1),8:(r+1,c),9:(r+1,c+1)
	}
    deltas = []
    #邻接阵arov
    arov={}
    for m in range(1,10):
        if arounddic[m] in nodelist:
            arov[m] = nodelist[arounddic[m]][7]
        else:
            arov[m] = 0
        delta =nodelist[(r,c)][7] - arov[m] 
        deltas.append(delta)
    B = np.cumsum(deltas)
    if B[-1]!= 0:
        rnd = random.random()*B[-1]
        ind = np.nonzero(B<rnd)
        if len(ind[0])==0:
            pos = 1
        else:
            pos = ind[0][-1]+2
    else:#九格同
        pos = 5 
        while pos == 5:
            pos = random.randint(1,9)
		#pos = 1 #左下
		#pos = 9 #右上
    #得到位置
    return arounddic[pos]        

def caculate(nodelist,radius):
    w = []
    delta  = radius
	#delta = radius/2#qqq
    for key in nodelist:
        w.append([nodelist[key][5],nodelist[key][7]])
    w.sort()
    w = np.array(w)
    R = np.arange(0,np.max(w[:,0]),delta)
    RL = np.zeros((len(R),2)) 
    RL[:,0] = R
    for i in range(len(RL)):
        RL[i,1] = np.sum(w[w[:,0]<RL[i,0],1])
    Rrho = np.zeros((len(R),2))
    Rrho[:,0] = R + delta/2
    for i in range(len(Rrho)-1):
        Rrho[i,1] = (RL[i+1,1] - RL[i,1])/(math.pi*(delta**2+2*delta*RL[i,0]))
    a = Rrho
    xx = np.nonzero(a[:,1]==0) 
    if len(xx[0])==0:
        x = 0 
    else:
        x = xx[0][0]
    Rrhox = Rrho[:x,:x]
    return Rrhox

def save(Rx,C,radius,maxnode,h=3,i=1):#i 预留次数参数
    se = {}
    for i in range(1,len(Rx)+1):
        se[i] = pd.Series(Rx[i])
    df = pd.concat([se[i] for i in range(1,len(Rx)+1) ],axis = 1)
    name = "C"+str(C)+"--r"+str(radius)+"--maxnode"+str(maxnode)+"--h"+str(h)+"--i"+str(i)
    df.to_csv(name+".csv")
    print("Winner winner chicken dinner")

def side_view(nodelist):
    h = []
    for k in nodelist:
        h.append(nodelist[k][7])
    a = Counter(h)
    print(a)
	
def spatial_range(nodelist):
    x = []
    y = []
    for k in nodelist.keys():
        x.append(k[0])
        y.append(k[1])
    return max(x),min(x),max(y),min(y)

def paint(nodelist):
	x = nodelist
	a = []
	m = []
	n = []
	c = []
	xmax,xmin,ymax,ymin = spatial_range(x)
	for k,v in x.items():
		a.append(k)
		c.append(1/(v[7]))
	for i in a:
		m.append(i[0])
		n.append(i[1])
	plt.figure(figsize=(10,10),dpi=100)
	plt.scatter(m,n,s = 0.1,c = c)
	plt.xlim((xmin,xmax))
	plt.ylim((ymin,ymax))
	plt.show()

In [163]:
def step(p):
    C = 0.05
    radius = 10
    h = 3
    L = 10**5
    if p == 1:
        return initiate(radius,C,L)
    else:
        nodelist,idx,limitt,index2coor=step(p-1)
        return onestep(nodelist,index2coor,idx,radius,C,h,limitt,L)

In [166]:
class City(object):
    def __init__(self,maxP,H,C,L = 10**5,radius = 10):
        '''初始全局参数
        可变参数：maxP:最大P,H:沙堆阈值,C:引力c
        默认参数：L:边框半径,radius:作用半径
        '''
        self.maxP = maxP
        self.H,self.C,self.L= H,C,L
        self.radius = radius
    def initiate(radius,C,L):
        coordinate = np.ones(2)*L/2.0
        nodelist = {(coordinate[0],coordinate[1]):{5:0.0,7:1}} #5：到心距 7：点数
        index2coor = {1:(coordinate[0],coordinate[1])}
        idx = index.Index()
        idx.insert(1,list(np.r_[coordinate,coordinate]))                     
        limitt = np.r_[coordinate - radius,coordinate + radius] 
        return nodelist,idx,limitt,index2coor

    def choosenode(nodelist,limitt,C,index2coor):#?
        white = ( (limitt[2]-limitt[0])*(limitt[3]-limitt[1])-len(nodelist) )*C
        exist = len(index2coor)+len(nodelist)*C
        rnd = random.random()*( exist + white )
        if rnd>exist:
            while 1:
                ii = int( random.random()*(limitt[2]-limitt[0])   + limitt[0] )
                jj = int( random.random()*(limitt[3]-limitt[1])   + limitt[1] )
                if (ii,jj) not in nodelist:
                    return (ii,jj),0
        else:
            pos = random.randint(1,len(index2coor))#右闭
            return index2coor[pos],1

    def onestep(nodelist,index2coor,idx,radius,C,h,limitt,L):
        two,flag = choosenode(nodelist,limitt,C,index2coor)  
        i = two[0]
        j = two[1]
        if flag == 0:
            intersection = list(idx.intersection([i-radius,j-radius, i+radius,j+radius])) 
            if len(intersection) > 0:
                for key in intersection:
                    if np.linalg.norm(np.array([index2coor[key]]) - np.array([i,j])) <= radius:  
                        state = 1
                    else:
                        state = 0
                if state == 1:
                    nodelist,index2coor,idx,limitt = insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L)
            return nodelist,index2coor,idx,limitt      

        elif flag==1: 
            if nodelist[(i,j)][7]+1 <= h:#不溢
                nodelist[(i,j)][7] += 1
                index2coor[len(index2coor)+1]=(i,j)
            else:
                while ((i,j) in nodelist.keys()) and (nodelist[(i,j)][7]+1 > h) :
                    (i,j) = sandpile(i,j,h,nodelist)
                if (i,j) in nodelist.keys():#溢出到有点的格子
                    nodelist[(i,j)][7] += 1
                    index2coor[len(index2coor)+1]=(i,j)
                else:#溢出到空格子
                    nodelist,index2coor,idx,limitt  = insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L)
            return nodelist,index2coor,idx,limitt

    def insert_newpoint(i,j,nodelist,index2coor,idx,limitt,L):
        nodelist[(i,j)] = {5:np.linalg.norm(np.array([i,j])-L/2),7:1}
        index2coor[len(index2coor)+1]=(i,j)
        newpoint = [i,j]
        idx.insert( len(index2coor), np.r_[newpoint, newpoint] )  ###idx
        for x in range(2):
            limitt[x]   = round(min(limitt[x],newpoint[x]-radius))
            limitt[x+2] = round(max(limitt[x+2],newpoint[x]+radius))
        return nodelist,index2coor,idx,limitt

    def sandpile(i,j,h,nodelist):
        r = i
        c = j
        arounddic = {
        1:(r-1,c-1),2:(r -1,c),3:(r -1,c+1),
        4:(r,c-1),5:(r,c),6:(r,c+1),
        7:(r+1,c-1),8:(r+1,c),9:(r+1,c+1)
        }
        deltas = []
        #邻接阵arov
        arov={}
        for m in range(1,10):
            if arounddic[m] in nodelist:
                arov[m] = nodelist[arounddic[m]][7]
            else:
                arov[m] = 0
            delta =nodelist[(r,c)][7] - arov[m] 
            deltas.append(delta)
        B = np.cumsum(deltas)
        if B[-1]!= 0:
            rnd = random.random()*B[-1]
            ind = np.nonzero(B<rnd)
            if len(ind[0])==0:
                pos = 1
            else:
                pos = ind[0][-1]+2
        else:#九格同
            pos = 5 
            while pos == 5:
                pos = random.randint(1,9)
            #pos = 1 #左下
            #pos = 9 #右上
        #得到位置
        return arounddic[pos]        
    def step(self,p):
        if p == 1:
            return initiate(self.radius,self.C,self.L)
        else:
            nodelist,idx,limitt,index2coor=City.step(self,p-1)
            return onestep(nodelist,index2coor,idx,self.radius,self.C,self.H,limitt,self.L)

In [170]:
a = City(2,3,0.05)
a.step(a.maxP)

NameError: name 'radius' is not defined

In [102]:
class a(object):
    def __init__(self,b):
        global selfb 
        selfb =b 
    def ca():
        print(selfb)

In [104]:
demo = a(1)
selfb

1

## utils

In [108]:
def printInitiate():
    instance = createCity(self.maxP,self.H,self.C)
    idx,limitt,index2coor = instance.initiate()
    print(idx,limitt,index2coor )
def cacuDistribution(index2coor):
    '''根据生成顺序计算分布
    传入index2coor返回nodelist'''
    nodelist = {index2coor[0]:{1:0,2:0}}
    for i in index2coor:
        if i in nodelist.keys:
            nodelist[i][2]+=1
        else:
            nodelist[i]={1:np.linalg.norm(np.array(i)-index2coor[0]),2:1}
    return nodelist

In [66]:
index2coor = [(1,1),(1,2),(1,2)]
createCity.cacuDistribution(index2coor)

{(1, 1): {1: 0, 2: 1}, (1, 2): {1: 1.0, 2: 2}}

In [ ]:
for i in index2coor:
    if i in nodelist.keys:
        nodelist[i][2]+=1
    else:
        nodelist[i]={1:np.linalg.norm(np.array(i)-index2coor[0]),2:1}
return nodelist

In [38]:
a = createCity(1,2,3)
a.printInitiate()

{(50000.0, 50000.0): {5: 0.0, 7: 1}} <rtree.index.Index object at 0x0000021B3CA592B0> [49990. 49990. 50010. 50010.] {1: (50000.0, 50000.0)}


In [16]:
a.L

100000

1